<a href="https://colab.research.google.com/github/sagarshah11/Brain-Model-FS-and-RS-neurons/blob/main/Brain_Oscillations_CompiledMorph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install --force-reinstall netpyne neuron matplotlib
# !pip install neuron matplotlib


In [ ]:
pip install pandas==1.5.3 numpy==1.23.5 neuron==8.2.2 netpyne==1.0.5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# Define the directory where the .hoc file and likely the .mod files are located
mod_dir = '/content/drive/MyDrive/Brain Modeling FS RS Neurons/Cell Morphologies/'

os.chdir(mod_dir)

In [ ]:
ls

In [ ]:
!nrnivmodl mod2
print("Compilation attempt finished.")

In [ ]:
from netpyne import specs, sim
import matplotlib.pyplot as plt

netParams = specs.NetParams()
%matplotlib inline

In [ ]:
cellRule = netParams.importCellParams(label='RS', somaAtOrigin=True,
            conds={'cellType': 'RS', 'cellModel': 'HH_full'},
            fileName= 'emodels_hoc/cADpyr_L4TPC.hoc',
            cellName= 'cADpyr_L4TPC',
            cellInstance = False,
            cellArgs=[0, 'morphologies', 'I03481.asc'])

print("Cell import attempted.")

In [ ]:
netParams.cellParams['RS']['secs'].keys()

In [ ]:
## Population parameters
netParams.popParams['RS'] = {'cellType': 'RS', 'numCells': 10} #changed ratio RS:FS = 10:1 per Aric's sugggestion.
# netParams.popParams['FS'] = {'cellType': 'RS', 'numCells': 1} #changed ratio RS:FS = 10:1 per Aric's sugggestion.

# netParams.synMechParams['exc'] = {'mod': 'Exp2Syn', 'tau1': 0.5, 'tau2': 5.0, 'e': 0} #updated tau1 to 0.5 per Markram et al. (1998)
# netParams.synMechParams['inh'] = {'mod': 'Exp2Syn','tau1':0.1, 'tau2':2.0, 'e': -80}  #updated tau2 to 2 ms per Wang & Buzsáki (1996)

netParams.stimSourceParams['IClamp'] = {'type': 'IClamp', 'delay': 550, 'dur': 600, 'amp': 0.1}  # Intracellular current step
netParams.stimTargetParams['IClamp->RS'] = {'source': 'IClamp', 'conds': {'cellType': 'RS'}, 'sec':'soma_0', 'loc': 0.5}  # Apply to the first RS cell
# netParams.stimTargetParams['IClamp->FS'] = {'source': 'IClamp', 'conds': {'cellType': 'FS'}, 'sec':'soma_0', 'loc': 0.5}  # Apply to the first RS cell

In [ ]:
simConfig = specs.SimConfig()       # object of class SimConfig to store simulation configuration

simConfig.duration = 1*1e3          # Duration of the simulation, in ms
simConfig.dt = 0.025                # Internal integration timestep to use
simConfig.verbose = False           # Show detailed messages
simConfig.
simConfig.recordTraces = {'V_soma':{'sec':'soma','loc':0.5,'var':'v'}}  # Dict with traces to record
simConfig.recordStep = 0.1          # Step size in ms to save data (e.g. V traces, LFP, etc)
simConfig.filename = 'tut2'         # Set file output name
simConfig.savePickle = False        # Save params, network and sim output to pickle file

simConfig.analysis['plotRaster'] = {'saveFig': True, 'showFig': True}                  # Plot a raster
simConfig.analysis['plotTraces'] = {'include': [0], 'saveFig': True, 'showFig' : True}  # Plot recorded traces for this list of cells
simConfig.analysis['plot2Dnet'] = {'saveFig': True, 'showFig' : True}                   # plot 2D cell positions and connections


In [ ]:
sim.initialize(
    simConfig = simConfig,
    netParams = netParams)  				# create network object and set cfg and net params
sim.net.createPops()               			# instantiate network populations


In [ ]:
sim.net.createCells()              			# instantiate network cells based on defined populations

In [ ]:
sim.net.connectCells()            			# create connections between cells based on params
sim.net.addStims() 							# add network stimulation

In [ ]:
sim.setupRecording()              			# setup variables to record for each cell (spikes, V traces, etc)
sim.runSim()                      			# run parallel Neuron simulation
sim.gatherData()                  			# gather spiking data and cell info from each node
sim.saveData()                    			# save params, cell info and sim output to file (pickle,mat,txt,etc)#
sim.analysis.plotData()         			# plot spike raster etc